In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [28]:
chrom_lengths = {'chr1': 247249719,
                 'chr2': 242951149,
                 'chr3': 199501827,
                 'chr4': 191273063,
                 'chr5': 180857866,
                 'chr6': 170899992,
                 'chr7': 158821424,
                 'chr8': 146274826,
                 'chr9': 140273252,
                 'chr10': 135374737,
                 'chr11': 134452384,
                 'chr12': 132349534,
                 'chr13': 114142980,
                 'chr14': 106368585,
                 'chr15': 100338915,
                 'chr16': 88827254,
                 'chr17': 78774742,
                 'chr18': 76117153,
                 'chr19': 63811651,
                 'chr20': 62435964,
                 'chr21': 46944323,
                 'chr22': 49691432}

In [37]:
endpoints = set([(k, v) for k, v in chrom_lengths.items()] + [(k, 0) for k in chrom_lengths.keys()])

with open('../data/DECODE/DECODE_Recombination_Events_Paternal_01OCT2013.txt', 'r') as f:
    next(f) # skip header
    for line in f:
        pieces = line.strip().split()
        if pieces[0] != 'chrX':
            endpoints.add((pieces[0], int(pieces[1])-1))
            endpoints.add((pieces[0], int(pieces[1])))
            endpoints.add((pieces[0], int(pieces[2])-1))
            endpoints.add((pieces[0], int(pieces[2])))

endpoints = sorted(endpoints)
endpoint_to_index = dict([(x, i) for i, x in enumerate(endpoints)])
print('endpoints', len(endpoints))

endpoints 616786


In [41]:
recomb_prob = np.zeros((len(endpoints),))
with open('../data/DECODE/DECODE_Recombination_Events_Paternal_01OCT2013.txt', 'r') as f:
    next(f) # skip header
    for line in f:
        pieces = line.strip().split()
        if pieces[0] != 'chrX':
            start_pos, end_pos = int(pieces[1]), int(pieces[2])
            recomb_prob[endpoint_to_index[(pieces[0], start_pos)]:(endpoint_to_index[(pieces[0], end_pos)])] += 1/(end_pos-start_pos)

#recomb_prob = recomb_prob/41745 # from paper - the number of meioses
recomb_prob = recomb_prob/30184

In [42]:
genome_length = sum(chrom_lengths.values())
print('genome length', genome_length)

genome length 2867732772


In [43]:
current_spot = 0
for i in np.argsort(recomb_prob[:-1]):
    if endpoints[i][0] == endpoints[i+1][0]:
        # not in a chromosome break
        current_spot += endpoints[i+1][1]-endpoints[i][1]
        if current_spot > genome_length/2:
            print(np.log10(recomb_prob[i]))
            break

-8.578163698218093


In [10]:
np.median(recomb_prob)

1.512832623158528e-08